In [ ]:
import os
import kaggle as kg
from kaggle.api.kaggle_api_extended import KaggleApi
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import RMSprop

In [ ]:
os.environ["KAGGLE_USERNAME"] = "dineshsingh09"
os.environ["KAGGLE_KEY"] = "d0c36def95197970dd807d8941622709"


In [ ]:
api = KaggleApi()

In [ ]:
kg.api.authenticate()

In [ ]:
kg.api.dataset_download_files(dataset="medahmedkrichen/devanagari-handwritten-character-datase",
                              path="dataset",unzip=True)

In [ ]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.png"):

            img_path.append(str(single_class_img_path))
            #print(str(single_class_img_path).split("/")[-2].split("_")[-1])
            img_label.append(str(single_class_img_path).split("/")[-2].split("_")[-1])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})        


In [ ]:
train_path = "dataset/DevanagariHandwrittenCharacterDataset/Train"
test_path = "dataset/DevanagariHandwrittenCharacterDataset/Test"

In [ ]:
training_data = train_test_df(train_path)
testing_data = train_test_df(test_path)

In [ ]:
training_data

In [ ]:
testing_data

In [ ]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))

In [ ]:

character2int

In [ ]:
training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                               inplace=True)

In [ ]:

training_data

In [ ]:
testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),
                     inplace=True)

In [ ]:

testing_data

In [ ]:
Y_true_train = to_categorical(y=training_data["label"],num_classes=46)

In [ ]:
Y_true_train

In [ ]:
Y_true_test = to_categorical(y=testing_data["label"],num_classes=46)

In [ ]:
Y_true_test

In [ ]:
def multiclass_dnn():

    input_to_dnn = Input(shape=(1024,))
    first_dense_out = Dense(units=1024,activation="relu") (input_to_dnn)
    output = Dense(units=46,activation="softmax") (first_dense_out)

    return Model(inputs=[input_to_dnn],outputs=[output])

In [ ]:
def custom_data_generator(data_df, mb_size):

    for time_step in range(data_df.shape[0]//mb_size):
        X_train_mb = list()

        for img_path in data_df.iloc[time_step*mb_size:(time_step+1)*mb_size,0]:

            img_np_array = plt.imread(img_path)
            reshaped_np_array = img_np_array.reshape(1024,)
            X_train_mb.append(reshaped_np_array)

        X_train_mb = np.array(X_train_mb)
        Y_train_mb = Y_true_train[time_step*mb_size:(time_step+1)*mb_size]

        yield X_train_mb, Y_train_mb


In [ ]:
mb_size = 782

In [ ]:
generator = custom_data_generator(training_data,mb_size)

In [ ]:
generator

In [ ]:
X_train_mb, Y_train_mb = generator.__next__()

In [ ]:
np.max(X_train_mb,axis=1)

In [ ]:
Y_train_mb.shape

In [ ]:
model = multiclass_dnn()

In [ ]:
model.summary()

In [ ]:
def loss_fn(Y_train_mb,Y_pred_mb):

    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=Y_train_mb,
                                                                   y_pred=Y_pred_mb))

In [ ]:
optimizer = RMSprop()

In [ ]:
epochs = 10
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (X_train_mb, Y_train_mb) in enumerate(generator):
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:
            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            Y_pred_mb = model(X_train_mb, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(Y_train_mb, Y_pred_mb)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Log every 50 batches.
        if step % 50 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1) * mb_size))